## Classifying DNA Sequences

This notebook will be very interesting for me personally. It will deal with a dataset, that I barely understand. We will explore the world of bioinformatics by using different common classifiers to classify DNA sequences, whether they are promoters or non-promoters. This project will use a dataset from the UCI Machine Learning Repository that has 106 DNA sequences.

#### Having a first look

Let's start importing the libraries and then the data.

In [1]:
import sys
import numpy as np
import sklearn
import pandas as pd

print('Python: {}'.format(sys.version))
print('Numpy: {}'.format(np.__version__))
print('Sklearn: {}'.format(sklearn.__version__))
print('Pandas: {}'.format(pd.__version__))

Python: 3.7.4 (v3.7.4:e09359112e, Jul  8 2019, 14:54:52) 
[Clang 6.0 (clang-600.0.57)]
Numpy: 1.18.3
Sklearn: 0.22.1
Pandas: 1.0.1


In [3]:
filePath = 'promoters.data'
names = ['Class', 'id', 'Sequence']
data = pd.read_csv(filePath, names = names)

The type of data that we are dealing with this time, is different compared to the other datasets. First of all, you see 3 columns:
* class: It is either `+` or `-`, where `+` indicates that the class is promoter, otherwise non-promoter. Promoters are DNA sequences, which serve as a kind of "On" switch to initiate the biological process of transcription for the genes, which follow the promoter DNA sequence. That is what Google gave me. For me, it means simply a special DNA sequence.
* id: the instance name
+ sequence: 57 sequential nucleotide ("base-pair") positions. Each position in the sequence is one of {a, g, t, c}. The .csv file is not clean yet, the sequence data contains some tabs, resulting in "\t", which need to be removed.

In [3]:
data

,Class,id,Sequence
0,+,S10,\t\ttactagcaatacgcttgcgttcggtggttaagtatgtataat...
1,+,AMPC,\t\ttgctatcctgacagttgtcacgctgattggtgtcgttacaat...
2,+,AROH,\t\tgtactagagaactagtgcattagcttatttttttgttatcat...
3,+,DEOP2,\taattgtgatgtgtatcgaagtgtgttgcggagtagatgttagaa...
4,+,LEU1_TRNA,\ttcgataattaactattgacgaaaagctgaaaaccactagaatgc...
...,...,...,...
101,-,799,\t\tcctcaatggcctctaaacgggtcttgaggggttttttgctga...
102,-,987,\t\tgtattctcaacaagattaaccgacagattcaatctcgtggat...
103,-,1226,\t\tcgcgactacgatgagatgcctgagtgcttccgttactggatt...
104,-,794,\t\tctcgtcctcaatggcctctaaacgggtcttgaggggtttttt...


In [4]:
print("We have {} promoters and {} non-promoters".format(len(data[data.Class == "+"]), len(data[data.Class == "-"])))

We have 53 promoters and 53 non-promoters


#### Pre-Processing

The idea of the course instructor is to put each column of the dataframe to a separated Pandas Series variable.

In [4]:
classes = data['Class']
sequences = list(data['Sequence'])

Each row of the dataset should contain all nucleotides and finally the corresponding class. While creating each row, the tab `\t` must be removed.

In [5]:
dataset = {}

for i, seq in enumerate(sequences):
    nucleotides = list(seq)
    nucleotides = [x for x in nucleotides if x != '\t']
    nucleotides.append(classes[i])
    dataset[i] = nucleotides

In [21]:
print(dataset[0])

['t', 'a', 'c', 't', 'a', 'g', 'c', 'a', 'a', 't', 'a', 'c', 'g', 'c', 't', 't', 'g', 'c', 'g', 't', 't', 'c', 'g', 'g', 't', 'g', 'g', 't', 't', 'a', 'a', 'g', 't', 'a', 't', 'g', 't', 'a', 't', 'a', 'a', 't', 'g', 'c', 'g', 'c', 'g', 'g', 'g', 'c', 't', 't', 'g', 't', 'c', 'g', 't', '+']


We put the dataset into a dataframe and realize that something looks different. Each row of the dataset is actually a column in the dataframe. That means, we need to transpose the dataframe.

In [6]:
dframe = pd.DataFrame(dataset)
dframe

,0,1,2,3,4,5,6,7,8,9,...,96,97,98,99,100,101,102,103,104,105
0,t,t,g,a,t,a,c,t,c,t,...,c,c,t,a,g,c,g,c,c,t
1,a,g,t,a,c,g,a,t,g,t,...,c,g,a,g,a,c,t,g,t,a
2,c,c,a,t,g,g,g,t,a,t,...,g,c,t,a,g,t,a,c,c,a
3,t,t,c,t,a,g,g,c,c,t,...,a,t,g,g,a,c,t,g,g,c
4,a,a,t,g,t,g,g,t,t,a,...,g,a,a,g,g,a,t,a,t,a
5,g,t,a,t,a,c,g,a,t,a,...,t,g,c,g,c,a,c,c,c,t
6,c,c,g,g,a,a,g,c,a,a,...,a,g,c,t,a,t,t,t,c,t
7,a,c,a,a,t,a,t,a,a,t,...,g,a,g,g,t,g,c,a,t,a
8,a,t,g,t,t,g,g,a,t,t,...,a,c,a,t,g,g,a,c,c,a
9,t,g,a,g,a,g,g,a,a,t,...,c,t,a,a,t,c,a,g,a,t


In [7]:
df = dframe.transpose()
df

,0,1,2,3,4,5,6,7,8,9,...,48,49,50,51,52,53,54,55,56,57
0,t,a,c,t,a,g,c,a,a,t,...,g,c,t,t,g,t,c,g,t,+
1,t,g,c,t,a,t,c,c,t,g,...,c,a,t,c,g,c,c,a,a,+
2,g,t,a,c,t,a,g,a,g,a,...,c,a,c,c,c,g,g,c,g,+
3,a,a,t,t,g,t,g,a,t,g,...,a,a,c,a,a,a,c,t,c,+
4,t,c,g,a,t,a,a,t,t,a,...,c,c,g,t,g,g,t,a,g,+
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101,c,c,t,c,a,a,t,g,g,c,...,g,a,a,c,t,a,t,a,t,-
102,g,t,a,t,t,c,t,c,a,a,...,t,c,a,a,c,a,t,t,g,-
103,c,g,c,g,a,c,t,a,c,g,...,a,a,g,g,c,t,t,c,c,-
104,c,t,c,g,t,c,c,t,c,a,...,a,g,g,a,g,g,a,a,c,-


The last column will be called "Class" for the clarity.

In [8]:
df.rename(columns = {57: 'Class'}, inplace = True) 
df[:5]

,0,1,2,3,4,5,6,7,8,9,...,48,49,50,51,52,53,54,55,56,Class
0,t,a,c,t,a,g,c,a,a,t,...,g,c,t,t,g,t,c,g,t,+
1,t,g,c,t,a,t,c,c,t,g,...,c,a,t,c,g,c,c,a,a,+
2,g,t,a,c,t,a,g,a,g,a,...,c,a,c,c,c,g,g,c,g,+
3,a,a,t,t,g,t,g,a,t,g,...,a,a,c,a,a,a,c,t,c,+
4,t,c,g,a,t,a,a,t,t,a,...,c,c,g,t,g,g,t,a,g,+


Obviously there are only 4 unique values for nucleotides: `t`, `a`, `c` and `g`. `freq` is the most common value (`top`) ’s frequency.

In [24]:
df.describe()

,0,1,2,3,4,5,6,7,8,9,...,48,49,50,51,52,53,54,55,56,Class
count,106,106,106,106,106,106,106,106,106,106,...,106,106,106,106,106,106,106,106,106,106
unique,4,4,4,4,4,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,2
top,t,a,a,c,a,a,a,a,a,a,...,c,c,c,t,t,c,c,c,t,-
freq,38,34,30,30,36,42,38,34,33,36,...,36,42,31,33,35,32,29,29,34,53


It would be nice to know, which value occurs how many times exactly. We can figure out the value counts for each column in the dataframe.

In [9]:
series = []
for name in df.columns:
    series.append(df[name].value_counts())
    
info = pd.DataFrame(series).transpose()
info

,0,1,2,3,4,5,6,7,8,9,...,48,49,50,51,52,53,54,55,56,Class
t,38.0,26.0,27.0,26.0,22.0,24.0,30.0,32.0,32.0,28.0,...,21.0,22.0,23.0,33.0,35.0,30.0,23.0,29.0,34.0,NaN
c,27.0,22.0,21.0,30.0,19.0,18.0,21.0,20.0,22.0,22.0,...,36.0,42.0,31.0,32.0,21.0,32.0,29.0,29.0,17.0,NaN
a,26.0,34.0,30.0,22.0,36.0,42.0,38.0,34.0,33.0,36.0,...,23.0,24.0,28.0,27.0,25.0,22.0,26.0,24.0,27.0,NaN
g,15.0,24.0,28.0,28.0,29.0,22.0,17.0,20.0,19.0,20.0,...,26.0,18.0,24.0,14.0,25.0,22.0,28.0,24.0,28.0,NaN
+,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,53.0
-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,53.0


I was really curious to see how the instructor deals with the character sequence data. The approach he chose was to apply the `get_dummies()` function, which treats the characters as categorical varibales, and then convert the categorical variables into dummy/indicator variables, hence numeric data.

In [10]:
numerical_df = pd.get_dummies(df)
numerical_df[:5]

,0_a,0_c,0_g,0_t,1_a,1_c,1_g,1_t,2_a,2_c,...,55_a,55_c,55_g,55_t,56_a,56_c,56_g,56_t,Class_+,Class_-
0,0,0,0,1,1,0,0,0,0,1,...,0,0,1,0,0,0,0,1,1,0
1,0,0,0,1,0,0,1,0,0,1,...,1,0,0,0,1,0,0,0,1,0
2,0,0,1,0,0,0,0,1,1,0,...,0,1,0,0,0,0,1,0,1,0
3,1,0,0,0,1,0,0,0,0,0,...,0,0,0,1,0,1,0,0,1,0
4,0,0,0,1,0,1,0,0,0,0,...,1,0,0,0,0,0,1,0,1,0


Having the columns `Class_+` and `Class_-` is redundant, as `Class_+` is already enough. It can be renamed to `Class`. 

In [11]:
df = numerical_df.drop(columns=['Class_-'])

df.rename(columns = {'Class_+': 'Class'}, inplace = True)
df[:5]

,0_a,0_c,0_g,0_t,1_a,1_c,1_g,1_t,2_a,2_c,...,54_t,55_a,55_c,55_g,55_t,56_a,56_c,56_g,56_t,Class
0,0,0,0,1,1,0,0,0,0,1,...,0,0,0,1,0,0,0,0,1,1
1,0,0,0,1,0,0,1,0,0,1,...,0,1,0,0,0,1,0,0,0,1
2,0,0,1,0,0,0,0,1,1,0,...,0,0,1,0,0,0,0,1,0,1
3,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,1,0,1,0,0,1
4,0,0,0,1,0,1,0,0,0,0,...,1,1,0,0,0,0,0,1,0,1


In [16]:
from sklearn import model_selection

# the 1 in drop() stands for column-wise drop
X = np.array(df.drop(['Class'], 1))
y = np.array(df['Class'])

seed = 1

X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.25, random_state=seed, shuffle=True)

print(len(X_train))
print(len(X_test))

79
27


## Step 3: Training and Testing the Classification Algorithms

The aim of the third step is: We will compare and compare the performance of different algorithms. It is actually a very superficial comparison, because we don't go in depth with none algorithm, we don't play with different hyperparameters to explore the potential of each classifier. But we can already see that DNN and SVM work the best, if we stick to the average validation accuracy score and the standard deviation. AdaBoost and Gaussian Process Classifier work fine too, but the standard deviation is relatively big.

In [17]:
from sklearn.neighbors import KNeighborsClassifier # KNN
from sklearn.neural_network import MLPClassifier # DNN
from sklearn.gaussian_process import GaussianProcessClassifier # Gaussian Process
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier # Decision Tree
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier # Random Forest, AdaBoost
from sklearn.naive_bayes import GaussianNB # Naive Bayes
from sklearn.svm import SVC #SVM

from sklearn.metrics import classification_report, accuracy_score

In [34]:
scoring = 'accuracy'
names = ["Nearest Neighbors", "Gaussian Process",
         "Decision Tree", "Random Forest", "Neural Net", "AdaBoost",
         "Naive Bayes", "SVM Linear", "SVM RBF", "SVM Sigmoid"]

classifiers = [
    KNeighborsClassifier(n_neighbors = 3),
    GaussianProcessClassifier(1.0 * RBF(1.0)),
    DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    MLPClassifier(alpha=1, max_iter=500), #L2 penalty / regularization
    AdaBoostClassifier(),
    GaussianNB(),
    SVC(kernel = 'linear'), 
    SVC(kernel = 'rbf'),
    SVC(kernel = 'sigmoid')
]

models = zip(names, classifiers)

results = []
names = []

for name, model in models:
    kfold = model_selection.KFold(n_splits=10, random_state = seed, shuffle=True)
    cv_results = model_selection.cross_val_score(model, X_train, y_train, cv=kfold, scoring=scoring)
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)

Nearest Neighbors: 0.810714 (0.099808)
Gaussian Process: 0.855357 (0.160605)
Decision Tree: 0.646429 (0.173279)
Random Forest: 0.671429 (0.148590)
Neural Net: 0.912500 (0.097628)
AdaBoost: 0.875000 (0.147902)
Naive Bayes: 0.837500 (0.112500)
SVM Linear: 0.912500 (0.097628)
SVM RBF: 0.875000 (0.111803)
SVM Sigmoid: 0.925000 (0.100000)


But how well would the classifiers perform on test data? Let's find it out.

It is interesting to observe that even the worse performing classifiers have a high precision score for the negative class as well as a high recall score for the positive class. Additionally: The Linear SVM has the highest test accuracy score and outperforms the neural network.

In [37]:
models = zip(names, classifiers)

for name, model in models:
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    print(name)
    print(accuracy_score(y_test, predictions))
    print(classification_report(y_test, predictions))

Nearest Neighbors
0.7777777777777778
              precision    recall  f1-score   support

           0       1.00      0.65      0.79        17
           1       0.62      1.00      0.77        10

    accuracy                           0.78        27
   macro avg       0.81      0.82      0.78        27
weighted avg       0.86      0.78      0.78        27

Gaussian Process
0.8888888888888888
              precision    recall  f1-score   support

           0       1.00      0.82      0.90        17
           1       0.77      1.00      0.87        10

    accuracy                           0.89        27
   macro avg       0.88      0.91      0.89        27
weighted avg       0.91      0.89      0.89        27

Decision Tree
0.8148148148148148
              precision    recall  f1-score   support

           0       1.00      0.71      0.83        17
           1       0.67      1.00      0.80        10

    accuracy                           0.81        27
   macro avg       0.8